In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 根据操作生成新的纪录

In [8]:
import pandas as pd
from pathlib import Path
import os
from tqdm import tqdm

In [3]:
from pathlib import Path
root = Path('../data')
root

PosixPath('../data')

In [4]:
ad_static = pd.read_csv(root/'ad_static.csv',low_memory=False, encoding='utf-8')
ad_operation = pd.read_csv(root/'ad_static_operation.csv', low_memory=False, encoding='utf-8')

In [5]:
ad_static.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
502148,82588,2019-03-19 03:56:04,21666,21917,18,198,64
502149,665036,2019-02-21 09:08:35,6184,11633,18,232,64
502150,491914,2019-01-25 05:01:10,1809,5278,5,232,64
502151,3990,2019-03-01 08:17:08,4255,3401,18,218,36
502152,222150,2019-02-28 07:56:53,4250,14527,18,94,36


In [6]:
ad_static.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502153 entries, 0 to 502152
Data columns (total 7 columns):
广告id      502153 non-null int64
创建时间      502153 non-null object
广告账户id    502153 non-null int64
商品id      502153 non-null int64
商品类型      502153 non-null int64
广告行业id    502153 non-null int64
素材尺寸      502153 non-null object
dtypes: int64(5), object(2)
memory usage: 26.8+ MB


In [7]:
ad_static.shape

(502153, 7)

In [8]:
ad_operation.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,2,100
1,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,3,"age:217,601,79,202,837,942,638,394,347,731,739..."
2,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-21 23:03:42,2,4,"281474976694272,281474976694272,28147497669427..."
3,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-22 23:37:02,1,1,0
4,347637,2019-02-21 23:03:42,18529,28775,13,12,34,2019-02-23 00:13:15,1,2,100


In [9]:
ad_operation.shape

(614182, 11)

In [10]:
ad_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614182 entries, 0 to 614181
Data columns (total 11 columns):
广告id       614182 non-null int64
创建时间       614182 non-null object
广告账户id     614182 non-null int64
商品id       614182 non-null int64
商品类型       614182 non-null int64
广告行业id     614182 non-null int64
素材尺寸       614182 non-null int64
创建/修改时间    614182 non-null object
操作类型       614182 non-null int64
修改字段       614182 non-null int64
操作后的字段值    614182 non-null object
dtypes: int64(8), object(3)
memory usage: 51.5+ MB


### 为ad_static增加三列： 人群定向 投放时段 广告状态

In [5]:
col_names = ad_static.columns.tolist()
col_names

['广告id', '创建时间', '广告账户id', '商品id', '商品类型', '广告行业id', '素材尺寸']

In [6]:
col_names.append('人群定向')
col_names.append('投放时段')
col_names.append('广告状态')
ad_static = ad_static.reindex(columns=col_names)
ad_static.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,人群定向,投放时段,广告状态
0,233649,2018-09-29 11:52:16,25681,7356,13,136,1,NaN,NaN,NaN
1,547531,2019-02-21 06:37:00,20696,-1,1,186,40,NaN,NaN,NaN
2,707841,2019-03-06 07:37:37,3968,-1,3,186,40,NaN,NaN,NaN
3,733436,2019-03-19 06:37:06,22405,31722,5,117,64,NaN,NaN,NaN
4,249105,2019-03-15 09:23:16,11360,29999,18,145,44,NaN,NaN,NaN


In [49]:
ad_static.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502153 entries, 0 to 502152
Data columns (total 10 columns):
广告id      502153 non-null int64
创建时间      502153 non-null object
广告账户id    502153 non-null int64
商品id      502153 non-null int64
商品类型      502153 non-null int64
广告行业id    502153 non-null int64
素材尺寸      502153 non-null object
人群定向      0 non-null float64
投放时段      0 non-null float64
广告状态      0 non-null float64
dtypes: float64(3), int64(5), object(2)
memory usage: 38.3+ MB


### 操作测试： 修改列名，定位id，根据广告id定位行

In [20]:
pd1_values = [0] * ad_static.shape[1]
pd1 = pd.DataFrame(pd1_values).T
pd1.columns = ad_static.columns
pd1

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,人群定向,投放时段,广告状态
0,0,0,0,0,0,0,0,0,0,0


In [21]:
pd1.iloc[0,:7] = ad_static.iloc[0, :7]
pd1

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,人群定向,投放时段,广告状态
0,233649,2018-09-29 11:52:16,25681,7356,13,136,1,0,0,0


In [91]:
curr_id = ad_operation.loc[0 ,'广告id']
curr_row = ad_static[ad_static.广告id== curr_id].index.tolist()[0]
curr_row

17

In [92]:
ad_static.iloc[17]

广告id                   347637
创建时间      2019-02-21 23:03:42
广告账户id                  18529
商品id                    28775
商品类型                       13
广告行业id                     12
素材尺寸                       34
人群定向                      NaN
投放时段                      NaN
广告状态                      NaN
Name: 17, dtype: object

### 对广告静态文件执行广告操作文件中的每一个操作

1. 操作类型1： 初始化属性
2. 操作类型2： 更改属性

In [9]:
for i in tqdm(range(ad_operation.shape[0])):
    # 得到广告id，和其在ad_static中的index
    curr_id = ad_operation.loc[i, '广告id']
    static_row = ad_static[ad_static.广告id == curr_id].index.tolist()[0]
    changedFeature = ' '
    if ad_operation.loc[i, '修改字段']==1:
        changedFeature = '广告状态'
    elif ad_operation.loc[i,'修改字段'] == 2:
        changedFeature = '出价'
    elif ad_operation.loc[i,  '修改字段'] == 3:
        changedFeature = '人群定向'
    else:
        changedFeature = '投放时段'
        
    # 初始化属性,并更新创建时间
    if ad_operation.loc[i, '操作类型'] == 1:    
        ad_static.loc[static_row, changedFeature] = ad_operation.loc[i, '操作后的字段值']
        ad_static.loc[static_row, '创建时间'] = ad_operation.loc[i, '创建/修改时间']
        #print('{}:  {} changed to {}'.format(curr_id, changedFeature, ad_operation.loc[i, '操作后的字段值']))
    # 修改属性 生成新的纪录
    else:       
        pd1_values = [0] * ad_static.shape[1]
        pd1 = pd.DataFrame(pd1_values).T
        pd1.columns = ad_static.columns
        pd1.iloc[0,:7] = ad_static.iloc[static_row, :7]
        pd1.loc[0, '创建时间'] = ad_operation.loc[i, '创建/修改时间']
        pd1.loc[0, changedFeature] = ad_operation.loc[i, '操作后的字段值']
        ad_static = ad_static.append(pd1, ignore_index=True, sort=False)
        #print(f'{curr_id}: appended')

100%|██████████| 614182/614182 [26:49:31<00:00,  3.49it/s]


In [11]:
ad_static.shape

(588901, 11)

In [14]:
ad_static.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,人群定向,投放时段,广告状态,出价
0,233649,2018-09-29 11:52:16,25681,7356,13,136,1,NaN,NaN,NaN,NaN
1,547531,2019-02-21 06:37:00,20696,-1,1,186,40,NaN,NaN,NaN,NaN
2,707841,2019-03-06 07:37:37,3968,-1,3,186,40,NaN,NaN,NaN,NaN
3,733436,2019-03-19 06:37:06,22405,31722,5,117,64,NaN,NaN,NaN,NaN
4,249105,2019-03-15 09:23:16,11360,29999,18,145,44,NaN,NaN,NaN,NaN


In [15]:
ad_static.to_csv('../data/ad_static_all_origin.csv', index=None, encoding='utf-8')

### 删除广告状态为0的广告id

In [ ]:
# def deleteADidStatus0(df):
#     ret, l = [], len(df)
#     for i in range(l):
#         if df.loc[i, '广告状态'] == 0:
#             ret.append(i)
#     # 删除行
#     new_df = df.drop(ret, axis=0)
#     df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
#     return new_df

In [ ]:
#new_ad_static = deleteADidStatus0(ad_static)

In [ ]:
#new_ad_static.shape

In [ ]:
#ad_static.head()

### 删除只有修改操作，没有创建操作的广告id

In [128]:
# 测试 提取广告id和操作类型 组成新的dataframe
testpd = pd.DataFrame(ad_operation.iloc[:100], columns =['广告id', '操作类型'])
testpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
广告id    100 non-null int64
操作类型    100 non-null int64
dtypes: int64(2)
memory usage: 1.6 KB


In [129]:
testpd['操作类型'] = testpd['操作类型'].astype('str')
testpd.head()
testpd.info()

,广告id,操作类型
0,347637,2
1,347637,2
2,347637,2
3,347637,1
4,347637,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
广告id    100 non-null int64
操作类型    100 non-null object
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [130]:
testpd = pd.pivot_table(testpd, index=['广告id'], aggfunc='sum')
testpd = testpd.reset_index()
testpd

,广告id,操作类型
0,281688,22211111111111111111111111111111111111
1,288207,2221111
2,327801,2221111111111111111111
3,347637,2221111111111111111111111
4,449295,22211111


In [131]:
testpd = testpd.append(pd.DataFrame({'广告id':145451,'操作类型':'223333'},pd.Index(['5'])))
testpd = testpd.reset_index()

In [132]:
testpd

,index,广告id,操作类型
0,0,281688,22211111111111111111111111111111111111
1,1,288207,2221111
2,2,327801,2221111111111111111111
3,3,347637,2221111111111111111111111
4,4,449295,22211111
5,5,145451,223333


In [133]:
ret = []
for i in range(len(testpd)):
    ops = testpd.loc[i, '操作类型']
    if '1' not in ops:
        ret.append(i)
ret

[5]

In [135]:
# 使用字典
# op_dict = {}
# retid, l = [], len(new_ad_static)
# for i in tqdm(range(l)):
#     if df.loc[i,'广告id'] in op_dict.keys():
#         op_dict[df.loc[i, '广告id']].append(df.loc[i, '广告状态'])
#     else:
#         op_dict[df.loc[i, '广告id']] = [df.loc[i, '广告状态']]
# print(len(op_dict.keys())
# for key in op_dict.keys():
#       if 1 not in op_dict[key]:
#           retid.append(key)
# print(len(retid))
# ret = new_ad_static[new_ad_static.广告id in retid].index.tolist()
      
# 优化，使用pandas操作
temp = ad_operation[['广告id', '操作类型']]
temp['操作类型'] = temp['操作类型'].astype('str')
temp = pd.pivot_table(temp, index=['广告id'], aggfunc='sum')
temp = temp.reset_index()
retid = []
for i in tqdm(range(len(temp))):
    ops = temp.loc[i, '操作类型']
    if '1' not in ops:
        retid.append(temp.loc[i, '广告id'])
print(len(retid), retid)

/home/ren/anaconda3/envs/phi/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 29647/29647 [00:00<00:00, 137649.92it/s]

274 [3230, 12295, 13703, 17420, 31228, 34080, 40481, 44528, 46127, 49849, 51072, 52044, 58649, 62740, 65467, 68255, 75021, 75370, 75800, 78055, 81107, 84616, 85815, 87117, 88292, 91653, 93998, 96575, 100738, 100830, 109347, 109739, 112747, 116744, 119893, 123461, 126242, 129220, 130000, 132454, 133399, 136665, 137700, 143637, 146411, 150825, 150994, 151787, 153288, 162264, 171906, 175211, 176106, 178318, 179766, 180773, 180817, 181226, 183547, 184191, 195469, 209395, 211284, 213508, 215744, 215963, 217972, 222852, 228518, 229183, 232221, 233203, 236495, 242665, 250598, 257105, 258928, 261296, 266435, 271703, 276078, 278853, 283190, 283932, 290542, 291714, 298905, 314404, 317118, 317783, 318045, 318291, 318299, 320981, 323188, 324000, 331331, 333534, 338500, 341894, 344122, 344142, 345669, 346255, 349041, 350865, 351329, 353664, 354334, 354913, 357380, 358512, 360274, 365441, 368294, 368411, 369145, 370143, 370194, 370338, 370640, 380234, 383289, 383983, 385916, 387608, 389028, 389541, 

In [136]:
ret = []

for j in tqdm(range(len(ad_static))):
    if ad_static.loc[j, '广告id'] in retid:
        ret.append(j)
new_ad_static = ad_static.drop(ret, axis=0)
new_ad_static = new_ad_static.reset_index()

100%|██████████| 588901/588901 [00:08<00:00, 68411.36it/s]


In [137]:
new_ad_static.shape

(587805, 12)

In [139]:
new_ad_static.tail()

,index,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,人群定向,投放时段,广告状态,出价
587800,588896,258747,2018-12-18 12:50:12,20545,3895,13,146,30,"age:217,601,79,202,837,942,638,394,347,731,739...",0,0,0
587801,588897,258747,2018-12-18 12:50:12,20545,3895,13,146,30,0,"281474976645120,281474976645120,28147497664512...",0,0
587802,588898,412811,2019-02-21 21:39:18,17804,26944,13,12,34,0,0,0,200
587803,588899,412811,2019-02-21 21:39:18,17804,26944,13,12,34,"age:217,601,79,202,837,942,638,394,347,731,739...",0,0,0
587804,588900,412811,2019-02-21 21:39:18,17804,26944,13,12,34,0,"281474976694272,281474976694272,28147497669427...",0,0


In [140]:
new_ad_static.to_csv('../data/ad_static_all.csv', index=None, encoding='utf-8')